In [1]:
from deoldify.device_id import DeviceId
from deoldify import device
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)
from deoldify.visualize import *
plt.style.use('dark_background')
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")
render_factor=35
from deoldify.visualize import get_image_colorizer
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time

# 假设ModelImageVisualizer已经被正确修改并且已经加载
visualizer = get_image_colorizer(render_factor=render_factor, artistic=True)


Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


/home/hasar/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hasar/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/hasar/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
import sys
import os
alike_module_path = os.path.abspath(os.path.join(os.getcwd(), '../ALIKE/'))
sys.path.append(alike_module_path)

from alike import ALike, configs

# if __name__ == '__main__':
# logging.basicConfig(level=logging.INFO)

# 定义直接在代码中设置的参数
input_path = ''  # 图片目录、电影文件或"camera0"
model_config = 'alike-l'  # 模型配置
device = 'cuda'  # 运行设备
top_k = -1  # 检测顶部K个关键点
scores_th = 0.4  # 检测器分数阈值
n_limit = 800  # 被检测的最大关键点数
output_dir = './keypoints'  # 关键点的输出目录

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# image_loader = ImageLoader(input_path)
model_alike = ALike(**configs[model_config],
                device=device,
                top_k=top_k,
                scores_th=scores_th,
                n_limit=n_limit)

Loaded model parameters from /home/hasar/dev/CARN-pytorch/ALIKE/models/alike-l.pth
Number of model parameters: 653.984KB


/home/hasar/.local/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
import open3d as o3d
import os
import json
import time
import importlib
from collections import OrderedDict
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
from PIL import Image
import argparse
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
import matplotlib.patches as patches
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../SuperPointPretrainedNetwork')))
from superpoint import SuperPointFrontend
import copy
import glob
import logging
import argparse
import numpy as np
from tqdm import tqdm
import open3d as o3d  
import json
import time
def tensor_to_np(tensor):
    tensor = tensor.cpu()
    np_img = tensor.mul(255).clamp(0, 255).byte().permute(1, 2, 0).numpy()
    return np_img

def sample(net, device, lr_image, scales):
    results = {}
    for scale in scales:
        lr = lr_image.unsqueeze(0).to(device)
        t1 = time.time()
        sr = net(lr, scale).detach().squeeze(0)
        t2 = time.time()
        sr_np = tensor_to_np(sr)
        results[scale] = sr_np
        print(f"Scale {scale}, Time taken: {t2-t1:.3f}s")
    return results

def get_SR(img):
   
    # lr_image = Image.open(img_path).convert("RGB")
    lr_image_tensor = transform(img)

    scales = [2, 4]
    sr_images = sample(net, device, lr_image_tensor, scales)
    print(img.shape)
    width = int(img.shape[0] * 0.5)
    height = int(img.shape[1] * 0.5)
    dim = (width, height)
        
    return sr_images, np.array(img)
    # return  np.array(img)



ckpt_path = '../checkpoint/carn.pth'
signal_dir = './dataset/data_3rd/signal/'
range_dir  = './dataset/data_3rd/range/'
pcd_dir = './dataset/data_3rd/pcd/'  
output_dir = 'kp_0m_15'
group = 1
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


module = importlib.import_module("model.{}".format('carn'))
net = module.Net(multi_scale=True, group=group)

state_dict = torch.load(ckpt_path)
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    new_state_dict[k] = v

net.load_state_dict(new_state_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)
transform = transforms.Compose([transforms.ToTensor()])




# for i in range(2376):  # 假设ID从0到1463

#     image_path = os.path.join(image_dir, f'image_{i}.jpg')
#     pcd_path = os.path.join(pcd_dir, f'pointcloud_{i}.pcd')
#     output_path = os.path.join(output_dir, f'keypoint_{i}.pcd')


    
    
#     img = Image.open(image_path).convert("RGB")
#         # sr_images, 
#     sr_images, ori_img = get_SR(img)
#     SR_4t, SR_2t = sr_images[4], sr_images[2]
#     print(ori_img.shape, SR_4t.shape, SR_2t.shape)

#     # 进行Gamma校正

#     gamma = 0.8
#     t0 = time.time()
#     Ori_img = np.power(ori_img / 255.0, gamma) * 255
#     Ori_img = np.clip(Ori_img, 0, 255).astype('uint8')
#     Ori_img_color =  visualizer.plot_transformed_image_from_array(Ori_img, watermarked = False,post_process = False, render_factor= 15)
#     t1 = time.time()
#     print(t1-t0)
    
    
#     SR_2t = np.power(SR_2t / 255.0, gamma) * 255
#     SR_2t = np.clip(SR_2t, 0, 255).astype('uint8')
#     SR_2t_color =   visualizer.plot_transformed_image_from_array(SR_2t, render_factor= 11)
    

    
#     # SR_4t = np.power(SR_4t / 255.0, gamma) * 255
#     # SR_4t = np.clip(SR_4t, 0, 255).astype('uint8')
#     # SR_4t_color = visualizer.plot_transformed_image_from_array(SR_4t, render_factor= 11)
    
#     keypoints_counts = []  # 临时存储当前迭代中每个图像的关键点数量
#     images = [
#             Ori_img_color,
#             # SR_2t, 
#             SR_2t_color,
#             ]

#     for idx, img in enumerate(images):
#             # print(img.shape)
#             # R, G, B = img[:,:,0], img[:,:,1], img[:,:,2]
#             # img = 0.2989 * R + 0.5870 * G + 0.1140 * B

#         img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         pred = model_alike(img_rgb, sub_pixel=True)
#         kpts = pred['keypoints']
#         kpts_int = np.round(kpts).astype(int)
#         print(len(kpts_int))
            
#         # fig, ax = plt.subplots(1)
#         # ax.imshow(img)
#         # for kp in kpts_int:
#         #     circ = patches.Circle((kp[0], kp[1]), 1, edgecolor='r', facecolor='none')
#         #     ax.add_patch(circ)
            
#         # plt.title(f"Keypoints Visualization on Image {img.shape}")
#         # plt.axis('off')
#         # plt.show()
        
#             # pcd = o3d.io.read_point_cloud(pcd_path)

#         pcd = o3d.io.read_point_cloud(pcd_path)
#         # 将点云转换为NumPy数组
#         points = np.asarray(pcd.points)

#         # 点云的高度和宽度（根据实际的点云数据进行调整）
#         height = 128
#         width = 1024

#         # 重塑为二维数组（高度，宽度，点的维度）
#         data = points.reshape((height, width, -1))  # 如果点云为XYZ，最后一维是3
        
#         offsets = [(dx, dy) for dx in range(-2, 3) for dy in range(-2, 3) if 0 <= dx < width and 0 <= dy < height]
#         keypoint_area_pcds = []

#         for x, y in kpts_int:
#             valid_coords = [(x + dx, y + dy) for dx, dy in offsets if 0 <= x + dx < width and 0 <= y + dy < height]
#             if valid_coords:
#                 # 使用 NumPy 高效索引
#                 nx, ny = zip(*valid_coords)
#                 keypoint_area_pcds.extend(data[ny, nx])

#         # 转换回Open3D的点云格式并保存
#         keypoint_cloud = o3d.geometry.PointCloud()
#         keypoint_cloud.points = o3d.utility.Vector3dVector(keypoint_area_pcds)
#         o3d.io.write_point_cloud(output_path, keypoint_cloud)
        
            
            
            
    # # 加载图像并检测关键点
    # img = cv2.imread(image_path)
    # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # pred = model(img_rgb)
    # kpts = pred['keypoints']

    # # 加载对应的PCD文件
    # pcd = o3d.io.read_point_cloud(pcd_path)


    # # 将点云转换为NumPy数组
    # points = np.asarray(pcd.points)

    # # 点云的高度和宽度（根据实际的点云数据进行调整）
    # height = 128
    # width = 1024

    # # 重塑为二维数组（高度，宽度，点的维度）
    # data = points.reshape((height, width, -1))  # 如果点云为XYZ，最后一维是3
    
    # # 对于每个关键点，四舍五入坐标，提取点云数据
    # keypoint_area_pcds = []
    # for x, y in kpts:
    #     x_round = int(round(x))
    #     y_round = int(round(y))
    #     for dx in range(-2, 2):  # 4x4区域，考虑边界条件
    #         for dy in range(-2, 2):
    #             nx, ny = x_round + dx, y_round + dy
    #             if 0 <= nx < width and 0 <= ny < height:
    #                 keypoint_area_pcds.append(data[ny, nx])
    
    # # 将关键点区域的点云数据转换回Open3D的点云格式并保存
    # keypoint_cloud = o3d.geometry.PointCloud()
    # keypoint_cloud.points = o3d.utility.Vector3dVector(keypoint_area_pcds)
    # o3d.io.write_point_cloud(output_path, keypoint_cloud)

    # print(f'Processed {i+1}/1464, Detected keypoints: {len(keypoint_area_pcds)}')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import open3d as o3d

def adjust_gamma(image, gamma=1.0):
    # 建立一个映射表
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")

    # 应用gamma校正使用映射表
    return cv2.LUT(image, table)


def remove_duplicates_ndarray(arrays):
    seen = set()
    unique_arrays = []
    for arr in arrays:
        # 将numpy数组转换为元组
        tup = tuple(arr.tolist())
        if tup not in seen:
            unique_arrays.append(arr)  # 添加原始numpy数组
            seen.add(tup)
    return unique_arrays


# 假设model_alike是你的关键点检测模型，pcd_path和output_path已经定义好
def detect_keypoints_and_save_pcd(range_img,
        ori_sgn_img,
        Ori_sgn_color,
        sng_2t,
        sng_2t_color, model_alike, pcd_path, output_path):
    
    keypoints_all_images = []
    keypoints_counts = 0

    range_img = cv2.cvtColor(range_img, cv2.COLOR_BGR2RGB)
    pred_range = model_alike(range_img, sub_pixel=True)
    kpts_range = pred_range['keypoints']
    kpts_range = np.round(kpts_range).astype(int)
    keypoints_counts = keypoints_counts + len(kpts_range)
    keypoints_all_images.extend(kpts_range)
    
    ori_sgn_img = cv2.cvtColor(ori_sgn_img, cv2.COLOR_BGR2RGB)
    pred_signal = model_alike(ori_sgn_img, sub_pixel=True)
    kpts_signal = pred_signal['keypoints']
    kpts_signal = np.round(kpts_signal).astype(int)
    # keypoints_counts.append(len(kpts_signal))
    keypoints_counts = keypoints_counts + len(kpts_signal)
    keypoints_all_images.extend(kpts_signal)
    
    Ori_sgn_color = cv2.cvtColor(Ori_sgn_color, cv2.COLOR_BGR2RGB)
    pred_sgn_color = model_alike(Ori_sgn_color, sub_pixel=True)
    kpts_sgn_color = pred_sgn_color['keypoints']
    kpts_sgn_color = np.round(kpts_sgn_color).astype(int)
    keypoints_counts = keypoints_counts + len(kpts_sgn_color)
    keypoints_all_images.extend(kpts_sgn_color)
    
    sng_2t = cv2.cvtColor(sng_2t, cv2.COLOR_BGR2RGB)
    pred_sgn_2t = model_alike(sng_2t, sub_pixel=True)
    kpts_sgn_2t = pred_sgn_2t['keypoints']
    kpts_sgn_2t[:, 0] = kpts_sgn_2t[:, 0] * 0.5
    kpts_sgn_2t[:, 1] = kpts_sgn_2t[:, 1] * 0.5
    kpts_sgn_2t = np.round(kpts_sgn_2t).astype(int)
    keypoints_counts = keypoints_counts + len(kpts_sgn_2t)
    keypoints_all_images.extend(kpts_sgn_2t)
    
    sng_2t_color = cv2.cvtColor(sng_2t_color, cv2.COLOR_BGR2RGB)
    pred_sgn_2t_color = model_alike(sng_2t_color, sub_pixel=True)
    kpts_sgn_2t_color = pred_sgn_2t_color['keypoints']
    kpts_sgn_2t_color[:, 0] = kpts_sgn_2t_color[:, 0] * 0.5
    kpts_sgn_2t_color[:, 1] = kpts_sgn_2t_color[:, 1] * 0.5
    kpts_sgn_2t_color = np.round(kpts_sgn_2t_color).astype(int)
    keypoints_counts = keypoints_counts + len(kpts_sgn_2t_color)
    keypoints_all_images.extend(kpts_sgn_2t_color)
    

    print(f"before keypoints average: {keypoints_counts}")
    keypoints_all_images = remove_duplicates_ndarray(keypoints_all_images)
    
    
    # # 调整关键点的尺寸映射
    # scale_x, scale_y = 128 / img.shape[0], 1024 / img.shape[1]
    # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # pred = model_alike(img_rgb, sub_pixel=True)
    # kpts = preadmin
    # d['keypoints']
    

    # 如果是第二张图片（SR_2t_color），需要调整关键点坐标
    # if scale_x != 1 and scale_y != 1:
    #     kpts[:, 0] = kpts[:, 0] * scale_x
    #     kpts[:, 1] = kpts[:, 1] * scale_y

    # kpts_int = np.round(kpts).astype(int)
    # keypoints_counts.append(len(kpts_int))
    # keypoints_all_images.extend(kpts_int)

# 打印关键点的平均数
    print(f"Ori_img_color and SR_2t_color keypoints average: {keypoints_counts}")

    # 以下是处理点云并保存的代码
    pcd = o3d.io.read_point_cloud(pcd_path)
    points = np.asarray(pcd.points)
    height, width = 128, 1024
    data = points.reshape((height, width, -1))
    
    keypoint_area_pcds = []
    for kpt in keypoints_all_images:
        x, y = kpt
        # for dx in range(-1, 2):
        #     for dy in range(-1, 2):
        if 0 <= x  < width and 0 <= y  < height:
            keypoint_area_pcds.append(data[y , x ])

    keypoint_cloud = o3d.geometry.PointCloud()
    keypoint_cloud.points = o3d.utility.Vector3dVector(keypoint_area_pcds)
    o3d.io.write_point_cloud(output_path, keypoint_cloud)

for i in range(1211):  # 假设ID从0到1463

    signal_path = os.path.join(signal_dir, f'image_{i}.jpg')
    range_path = os.path.join(range_dir, f'image_{i}.jpg')
    pcd_path = os.path.join(pcd_dir, f'pointcloud_{i}.pcd')
    output_path = os.path.join(output_dir, f'keypoint_{i}.pcd')

    signal_img = Image.open(signal_path).convert("RGB")
    range_img = Image.open(range_path).convert("RGB")
    signal_img = np.array(signal_img)
    range_img = np.array(range_img)
    sr_sgn_img, ori_sgn_img = get_SR(signal_img)
    sng_4t, sng_2t = sr_sgn_img[4], sr_sgn_img[2]
    print(ori_sgn_img.shape, sng_4t.shape, sng_2t.shape)

    # 进行Gamma校正

    range_img = adjust_gamma(range_img, gamma=3)
    sng_2t = adjust_gamma(sng_2t, gamma=1.5)
    ori_sgn_img = adjust_gamma(ori_sgn_img, gamma=1.5)
    
    Ori_sgn_color =  visualizer.plot_transformed_image_from_array(ori_sgn_img, watermarked = False,post_process = False, render_factor= 15)
    sng_2t_color = visualizer.plot_transformed_image_from_array(sng_2t, watermarked = False,post_process = False, render_factor= 15)
    
    images = [
        range_img,
        ori_sgn_img,
        Ori_sgn_color,
        sng_2t,
        sng_2t_color
        ]

    detect_keypoints_and_save_pcd(range_img,
        ori_sgn_img,
        Ori_sgn_color,
        sng_2t,
        sng_2t_color, model_alike, pcd_path, output_path)
    

# 注意：确保model_alike函数、pcd_path和output_path已经正确定义


Scale 2, Time taken: 0.156s
Scale 4, Time taken: 0.007s
(128, 1024, 3)
(128, 1024, 3) (512, 4096, 3) (256, 2048, 3)
before keypoints average: 424.0
Ori_img_color and SR_2t_color keypoints average: 424.0
Scale 2, Time taken: 0.002s
Scale 4, Time taken: 0.002s
(128, 1024, 3)
(128, 1024, 3) (512, 4096, 3) (256, 2048, 3)
before keypoints average: 422.2
Ori_img_color and SR_2t_color keypoints average: 422.2
Scale 2, Time taken: 0.006s
Scale 4, Time taken: 0.004s
(128, 1024, 3)
(128, 1024, 3) (512, 4096, 3) (256, 2048, 3)
before keypoints average: 421.0
Ori_img_color and SR_2t_color keypoints average: 421.0
Scale 2, Time taken: 0.002s
Scale 4, Time taken: 0.002s
(128, 1024, 3)
(128, 1024, 3) (512, 4096, 3) (256, 2048, 3)
before keypoints average: 415.0
Ori_img_color and SR_2t_color keypoints average: 415.0
Scale 2, Time taken: 0.003s
Scale 4, Time taken: 0.004s
(128, 1024, 3)
(128, 1024, 3) (512, 4096, 3) (256, 2048, 3)
before keypoints average: 418.2
Ori_img_color and SR_2t_color keypoints 